###### Initialize morpheus session

In [1]:
import org.opencypher.morpheus.api.MorpheusSession

implicit val morpheus: MorpheusSession = MorpheusSession.local()

Intitializing Scala interpreter ...

Spark Web UI available at http://262f2fcdc736:4040
SparkContext available as 'sc' (version = 2.4.2, master = local[*], app id = local-1586450928491)
SparkSession available as 'spark'


import org.opencypher.morpheus.api.MorpheusSession
morpheus: org.opencypher.morpheus.api.MorpheusSession = MorpheusSession


###### Retrieve environment variables

In [2]:
val NEO4J_SHOPS_URI = sys.env("NEO4J_SHOPS_URI")
val NEO4J_SHOPS_USERNAME = sys.env("NEO4J_SHOPS_USERNAME")
val NEO4J_SHOPS_PASSWORD = sys.env("NEO4J_SHOPS_PASSWORD")
val NEO4J_NEARBY_URI = sys.env("NEO4J_NEARBY_URI")
val NEO4J_NEARBY_USERNAME = sys.env("NEO4J_NEARBY_USERNAME")
val NEO4J_NEARBY_PASSWORD = sys.env("NEO4J_NEARBY_PASSWORD")
val NEO4J_COUPONS_URI = sys.env("NEO4J_COUPONS_URI")
val NEO4J_COUPONS_USERNAME = sys.env("NEO4J_COUPONS_USERNAME")
val NEO4J_COUPONS_PASSWORD = sys.env("NEO4J_COUPONS_PASSWORD")

NEO4J_SHOPS_URI: String = bolt://neo4j-shops:7687
NEO4J_SHOPS_USERNAME: String = neo4j
NEO4J_SHOPS_PASSWORD: String = shops
NEO4J_NEARBY_URI: String = bolt://neo4j-nearby:7687
NEO4J_NEARBY_USERNAME: String = neo4j
NEO4J_NEARBY_PASSWORD: String = nearby
NEO4J_COUPONS_URI: String = bolt://neo4j-coupons:7687
NEO4J_COUPONS_USERNAME: String = neo4j
NEO4J_COUPONS_PASSWORD: String = coupons


###### Define Neo4jContext class

In [3]:
import org.opencypher.okapi.neo4j.io.Neo4jConfig
import org.neo4j.driver.v1.{Driver, Session, StatementResult}

case class Neo4jContext(config: Neo4jConfig) {

    val driver = config.driver()
    val session = driver.session()

    def close(): Unit = {
        session.close()
        driver.close()
    }

    def execute(cypher: String): StatementResult = {
        session.run(cypher)
    }
}

import org.opencypher.okapi.neo4j.io.Neo4jConfig
import org.neo4j.driver.v1.{Driver, Session, StatementResult}
defined class Neo4jContext


###### Create Neo4jConfig objects, one for each neo4j instance

In [4]:
import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig

import java.net.URI
import org.opencypher.okapi.neo4j.io.Neo4jConfig


In [5]:
val neo4jShopsConfig = new Neo4jConfig(
    URI.create(NEO4J_SHOPS_URI),
    user = NEO4J_SHOPS_USERNAME, 
    password = Some(NEO4J_SHOPS_PASSWORD), 
    encrypted = false
)

neo4jShopsConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-shops:7687,neo4j,Some(shops),false,100000,100000,1000,10)


In [6]:
val neo4jNearbyConfig = new Neo4jConfig(
    URI.create(NEO4J_NEARBY_URI),
    user = NEO4J_NEARBY_USERNAME, 
    password = Some(NEO4J_NEARBY_PASSWORD), 
    encrypted = false
)

neo4jNearbyConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-nearby:7687,neo4j,Some(nearby),false,100000,100000,1000,10)


In [7]:
val neo4jCouponsConfig = new Neo4jConfig(
    URI.create(NEO4J_COUPONS_URI),
    user = NEO4J_COUPONS_USERNAME, 
    password = Some(NEO4J_COUPONS_PASSWORD), 
    encrypted = false
)

neo4jCouponsConfig: org.opencypher.okapi.neo4j.io.Neo4jConfig = Neo4jConfig(bolt://neo4j-coupons:7687,neo4j,Some(coupons),false,100000,100000,1000,10)


###### Create Neo4jContext objects, one for each Neo4jConfig object

In [8]:
val neo4jShopsContext = new Neo4jContext(neo4jShopsConfig)

neo4jShopsContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-shops:7687,neo4j,Some(shops),false,100000,100000,1000,10))


In [9]:
val neo4jNearbyContext = new Neo4jContext(neo4jNearbyConfig)

neo4jNearbyContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-nearby:7687,neo4j,Some(nearby),false,100000,100000,1000,10))


In [10]:
val neo4jCouponsContext = new Neo4jContext(neo4jCouponsConfig)

neo4jCouponsContext: Neo4jContext = Neo4jContext(Neo4jConfig(bolt://neo4j-coupons:7687,neo4j,Some(coupons),false,100000,100000,1000,10))


###### Initialize Neo4j graph with sample data

In [11]:
neo4jShopsContext.execute("MATCH (n) DETACH DELETE n")
neo4jShopsContext.execute(
    """|CREATE (:Person {name:'Alice'})-[:OWNS]->(:Shop {name:'Shop A'})
       |CREATE (:Person {name:'Bob'})-[: OWNS]->(:Shop {name:'Shop B'})""".stripMargin
)

res0: org.neo4j.driver.v1.StatementResult = org.neo4j.driver.internal.InternalStatementResult@51c017da


In [12]:
neo4jNearbyContext.execute("MATCH (n) DETACH DELETE n")
neo4jNearbyContext.execute(
    """|CREATE (a:Shop {name:'Shop A'})
       |CREATE (b:Shop {name:'Shop B'})
       |CREATE (:Person {name:'Diana'})-[:IS_NEAR_BY]->(a)
       |CREATE (:Person {name:'Eve'})-[:IS_NEAR_BY]->(b)
       |CREATE (:Person {name:'Carl'})-[:IS_NEAR_BY]->(a)""".stripMargin
)

res1: org.neo4j.driver.v1.StatementResult = org.neo4j.driver.internal.InternalStatementResult@7691d946


In [13]:
neo4jCouponsContext.execute("MATCH (n) DETACH DELETE n")
neo4jCouponsContext.execute(
    """|CREATE (:Person {name:'Alice'})-[:OFFERS]->(:Coupon {discount:'10%'})
       |CREATE (:Person {name:'Bob'})-[:OFFERS]->(:Coupon {discount:'Free coffee!'})""".stripMargin
)

res2: org.neo4j.driver.v1.StatementResult = org.neo4j.driver.internal.InternalStatementResult@2ac0ca61


###### Register Neo4j graphs into morpheus

In [14]:
import org.opencypher.morpheus.api.GraphSources
import org.opencypher.okapi.api.graph.Namespace

morpheus.registerSource(Namespace("gshops"), GraphSources.cypher.neo4j(neo4jShopsConfig))
morpheus.registerSource(Namespace("gnearby"), GraphSources.cypher.neo4j(neo4jNearbyConfig))
morpheus.registerSource(Namespace("gcoupons"), GraphSources.cypher.neo4j(neo4jCouponsConfig))

import org.opencypher.morpheus.api.GraphSources
import org.opencypher.okapi.api.graph.Namespace


###### Check the content of each graph

In [15]:
morpheus.cypher(
    """|FROM GRAPH gshops.graph
       |MATCH (n)
       |RETURN n""".stripMargin
  ).show

╔═══════════════════════════════╗
║ n                             ║
╠═══════════════════════════════╣
║ (:`Shop` {`name`: 'Shop A'})  ║
║ (:`Shop` {`name`: 'Shop B'})  ║
║ (:`Person` {`name`: 'Alice'}) ║
║ (:`Person` {`name`: 'Bob'})   ║
╚═══════════════════════════════╝
(4 rows)


In [16]:
morpheus.cypher(
    """|FROM GRAPH gnearby.graph
       |MATCH (n)
       |RETURN n""".stripMargin
  ).show

╔═══════════════════════════════╗
║ n                             ║
╠═══════════════════════════════╣
║ (:`Shop` {`name`: 'Shop A'})  ║
║ (:`Shop` {`name`: 'Shop B'})  ║
║ (:`Person` {`name`: 'Diana'}) ║
║ (:`Person` {`name`: 'Eve'})   ║
║ (:`Person` {`name`: 'Carl'})  ║
╚═══════════════════════════════╝
(5 rows)


In [17]:
morpheus.cypher(
    """|FROM GRAPH gcoupons.graph
       |MATCH (n)
       |RETURN n""".stripMargin
  ).show

╔══════════════════════════════════════════╗
║ n                                        ║
╠══════════════════════════════════════════╣
║ (:`Person` {`name`: 'Alice'})            ║
║ (:`Person` {`name`: 'Bob'})              ║
║ (:`Coupon` {`discount`: '10%'})          ║
║ (:`Coupon` {`discount`: 'Free coffee!'}) ║
╚══════════════════════════════════════════╝
(4 rows)


###### Query to retrieve the shoppers that have to be notified

In [18]:
morpheus.cypher(
    """|FROM GRAPH gshops.graph
       |MATCH (o1:Person)-[:OWNS]->(s1:Shop)
       |FROM GRAPH gcoupons.graph
       |MATCH (o2:Person)-[:OFFERS]->(c:Coupon)
       |WHERE o1.name = o2.name
       |FROM GRAPH gnearby.graph
       |MATCH (t:Person)-[:IS_NEAR_BY]->(s2:Shop)
       |WHERE s1.name = s2.name
       |RETURN t, c""".stripMargin
  ).show

╔═══════════════════════════════╤══════════════════════════════════════════╗
║ t                             │ c                                        ║
╠═══════════════════════════════╪══════════════════════════════════════════╣
║ (:`Person` {`name`: 'Diana'}) │ (:`Coupon` {`discount`: '10%'})          ║
║ (:`Person` {`name`: 'Eve'})   │ (:`Coupon` {`discount`: 'Free coffee!'}) ║
║ (:`Person` {`name`: 'Carl'})  │ (:`Coupon` {`discount`: '10%'})          ║
╚═══════════════════════════════╧══════════════════════════════════════════╝
(3 rows)
